In [1]:
import sys
sys.path.append('/home/xinyiz/pamrats')

import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim

# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import average_precision_score

import gae.gae.optimizer as optimizer
import gae.gae.model
import gae.gae.preprocessing as preprocessing

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN,MiniBatchKMeans,AgglomerativeClustering
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.image as mpimg


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
ifplot=True
ifcluster=True
byCT=True

logitL1=0.5 #smaller is sparser
geneThresh=2.5 #times std above mean

# inverseAct=None
plottype='umap'
pca=PCA()
minCells=15 #min number of cells for analysis
minCell_clusterDE=5
# clustermethod=['leiden']
clustermethod=['leiden','agglomerative','kmeanbatch']
#umap/leiden clustering parameters
n_neighbors=10
min_dist=0.25
n_pcs=40 #for clustering
# resolution=[0.5,0.8,1,1.5]
resolution=[0.05,0.1,0.2,0.3,0.5,0.8,1,1.5]
plotepoch=9990
savenameAdd=''
#DBscan
epslist= [6,8,10]
min_sampleslist=[15,30,45] 
#agglomerative
nclusterlist=[2,3,4,5,8,10,15]
aggMetric=['euclidean']


combineCelltype={'glia':['Astro','Micro', 'OPC', 'Oligo'],'CA':['CA1', 'CA2', 'CA3']}

use_cuda=True
fastmode=False #Validate during training pass
seed=3

protein=None #'scaled_binary'
# proteinWeights=0.05
# randFeatureSubset=None
plot_samples={'disease13':'AD_mouse9494','control13':'AD_mouse9498','disease8':'AD_mouse9723','control8':'AD_mouse9735'}
plot_sample_X=['scaled']
plotRecon='' #'meanRecon'
# plot_sample_X=['corrected','scaled']
standardizeX=False
name='secretoryExprs'
plotsavepath=os.path.join('/mnt/external_ssd/xinyi/plots/data/',name,plot_sample_X[0])
if not os.path.exists(plotsavepath):
    os.mkdir(plotsavepath)
datadir='/home/xinyiz/2021-01-13-mAD-test-dataset'

xcorrModel='logminmax'
nameModel='allk20XA_05_dca_over'
plotsavepathModel='/mnt/external_ssd/xinyi/plots/train_gae_starmap/'+nameModel

In [3]:
#Load data
savedir=os.path.join('/mnt/xinyi/','starmap')
adj_dir=os.path.join(savedir,'a')

featureslist={}
if plot_sample_X[0] in ['corrected','scaled']:
    scaleddata=scanpy.read_h5ad(datadir+'/2020-12-27-starmap-mAD-scaled.h5ad')
    
    for s in plot_samples.keys():
        if plot_sample_X[0]=='corrected':
            featureslist[s+'X_'+'corrected']=scaleddata.layers['corrected'][scaleddata.obs['sample']==plot_samples[s]]
        else:
            featureslist[s+'X_'+'scaled']=scaleddata.layers['scaled'][scaleddata.obs['sample']==plot_samples[s]]
    
scaleddata=scanpy.read_h5ad(datadir+'/2020-12-27-starmap-mAD-raw.h5ad')

for s in plot_samples.keys():
    scaleddata_train=scaleddata.X[scaleddata.obs['sample']==plot_samples[s]]

    if plot_sample_X[0]=='logminmax':
        featurelog_train=np.log2(scaleddata_train+1/2)
        scaler = MinMaxScaler()
        featurelog_train_minmax=np.transpose(scaler.fit_transform(np.transpose(featurelog_train)))
        featureslist[s+'X_'+plot_sample_X[0]]=torch.tensor(featurelog_train_minmax)
    featureslist[s+'X_'+'raw']=scaleddata_train

In [4]:
pathway=pd.read_csv('/home/xinyiz/secretory/QuickGO-annotations-0006887-exocytosis.tsv', sep='\t', header=0)
savename='exocytosis'
# plotname='secretion by cell '

In [5]:
starmapG=np.array(scaleddata.var.index)

In [6]:
pathwayG=np.array(pathway['SYMBOL'].str.upper())

In [7]:
geneNames,commstarmap,commPathway=np.intersect1d(starmapG,pathwayG,return_indices=True)

In [8]:
np.random.seed(seed)
def plotExprs(exprs,vmax,savename,embedding,savepath,plotname,savenameAdd=''):
    fig, ax = plt.subplots(dpi=400)
    ax.scatter(embedding[:,0],embedding[:,1],c=exprs,cmap='Greys',vmin=0,vmax=vmax,edgecolors='none',s=1.5,alpha=1)
    plt.gca().set_aspect('equal', 'datalim')
    fig.set_figheight(5)
    fig.set_figwidth(5)
    plt.title(plotname, fontsize=14)
    plt.savefig(os.path.join(savepath,savename+savenameAdd+'.jpg'))
#     plt.show()
    
#     fig.clf()
    plt.close('all')

In [9]:
np.random.seed(seed)
def plotExprs_plaque(exprs,vmax,savename,embedding,savepath,savenameAdd=''):
    imagepath=os.path.join(datadir,sampleidx,'trimmed_images','plaque.tif')
    image=mpimg.imread(imagepath,'tif')
    image=plt.imshow(image)
    plt.scatter(embedding[:,0]/0.3,embedding[:,1]/0.3,c=exprs,cmap='copper',vmin=0,vmax=vmax,edgecolors='none',s=1.5,alpha=1)
    plt.savefig(os.path.join(savepath,savename+savenameAdd+'.jpg'),dpi=1200)
#     plt.show()
    
#     fig.clf()
    plt.close('all')

In [10]:
#all cells
from PIL import Image  

Image.MAX_IMAGE_PIXELS = None

vmax=0
for s in plot_samples.keys():
    sampleidx=plot_samples[s]
    for xcorr in plot_sample_X:
        samplename=s+'X_'+xcorr
        exprs=featureslist[samplename][:,commstarmap]
       
        vmax=max(vmax,np.max(np.sum(exprs,axis=1)))

for s in plot_samples.keys():
    sampleidx=plot_samples[s]
    
    celltype_broad=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']
    celltype_sub=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'cell_type_label']
    region=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']
    sobj_coord_np=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,['x','y']].to_numpy()
    for xcorr in plot_sample_X:
        samplename=s+'X_'+xcorr
        exprs=featureslist[samplename][:,commstarmap]
       
        savedir=plotsavepath

#         plotExprs(np.sum(exprs,axis=1),vmax,savename+'_'+s,sobj_coord_np,savedir,plotname+s,savenameAdd=savenameAdd)
        savenameAdd='_plaqueOverlay'
        plotExprs_plaque(np.sum(exprs,axis=1),vmax,savename+'_'+s,sobj_coord_np,savedir,savenameAdd=savenameAdd)


In [8]:
def plotCTcomp(labels,samplenameList,exprs,exprs_raw,savepath,savenamecluster,addname='',exprsThresh=0):
    exprsMin=np.min(exprs)
    labelUnique=np.unique(labels)
    sampleUnique=np.unique(samplenameList)
    res=np.zeros((labelUnique.size,sampleUnique.size))
    resNCells=np.zeros((labelUnique.size,sampleUnique.size))
    for li in range(res.shape[0]):
        l=labelUnique[li]
        label_curr=(labels==l)
        for si in range(res.shape[1]):
            s=sampleUnique[si]
            sample_curr=(samplenameList==s)
            cellIdx=np.logical_and(label_curr,sample_curr)
            if np.sum(cellIdx)==0:
                res[li,si]=exprsMin
            else:
                res[li,si]=np.mean(exprs[cellIdx])
#                 print(exprs_raw[cellIdx])
                resNCells[li,si]=np.sum(exprs_raw[cellIdx]>=exprsThresh)/np.sum(cellIdx)
    
    res=(res-exprsMin)/(np.max(res)-exprsMin)
#     print(savenamecluster)
#     print(resNCells)
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(res,cmap='binary')
    fig.colorbar(im,orientation='vertical', shrink = 0.5)
    ax.set_yticks(np.arange(labelUnique.size))
    ax.set_yticklabels(labelUnique)
    ax.set_xticks(np.arange(sampleUnique.size))
    ax.set_xticklabels(sampleUnique)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
    fig.tight_layout()
    plt.savefig(os.path.join(savepath,savenamecluster+'_exprsByClusterSample'+addname+'.jpg'))
    plt.close()
    
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(resNCells,cmap='binary')
    fig.colorbar(im,orientation='vertical', shrink = 0.5)
    ax.set_yticks(np.arange(labelUnique.size))
    ax.set_yticklabels(labelUnique)
    ax.set_xticks(np.arange(sampleUnique.size))
    ax.set_xticklabels(sampleUnique)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
    fig.tight_layout()
    plt.savefig(os.path.join(savepath,savenamecluster+'_NonzeroCells'+addname+'.jpg'))
    plt.close()
        
        

def compLeidenPathway(ctlist,samplenameList,exprs,exprs_raw,n_neighbors,n_pcs,min_dist,resolution,addName=''):
    for r in resolution:
        savenamecluster='leiden_nn'+str(n_neighbors)+'mdist0'+str(int(min_dist*100))+'n_pcs'+str(n_pcs)+'res'+str(r)+'epoch'+str(plotepoch)
        readpath=os.path.join(clustersavedir,savenamecluster)
        if not os.path.exists(readpath):
            print('DNE: '+readpath)
            continue
        with open(readpath, 'rb') as input:
            labels = pickle.load(input)
        labels=np.array(labels)
        if np.unique(labels).shape[0]==1:
            continue
        
        savepath=os.path.join(clustersavedir,savenamecluster+'_'+name+'_'+xcorr)
        if not os.path.exists(savepath):
            os.mkdir(savepath)
        savepath=os.path.join(savepath,savename)
        if not os.path.exists(savepath):
            os.mkdir(savepath)
        
        plotCTcomp(labels,samplenameList,exprs,exprs_raw,savepath,'all',addName)
        for ct in np.unique(ctlist):
            ctidx=ctlist==ct
            plotCTcomp(labels[ctidx],samplenameList[ctidx],exprs[ctidx],exprs_raw[ctidx],savepath,ct,addName)

def compDBscan(ctlist,samplenameList,exprs,epsL,min_samplesL,n_pcs,addName=''):
    for eps in epsL:
        for min_samples in min_samplesL:
            savenamecluster='dbscan_eps'+str(eps)+'msamples'+str(min_samples)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
            readpath=os.path.join(clustersavedir,savenamecluster)
            if not os.path.exists(readpath):
                print('DNE: '+readpath)
                continue
            with open(readpath, 'rb') as input:
                labels = pickle.load(input)
            labels=np.array(labels)
            if np.unique(labels).shape[0]==1:
                continue
            
            savepath=clustersavedir
            plotCTcomp(labels,ctlist,savepath,savenamecluster,addName)
                
def compAgg(ctlist,nclusterL,aggmetricL,n_pcs,addName=''):
    for ncluster in nclusterL:
        for aggmetric in aggmetricL:
            savenamecluster='agg_ncluster'+str(ncluster)+aggmetric+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
            readpath=os.path.join(clustersavedir,savenamecluster)
            if not os.path.exists(readpath):
                print('DNE: '+readpath)
                continue
            with open(readpath, 'rb') as input:
                labels = pickle.load(input)
            labels=np.array(labels)
            if np.unique(labels).shape[0]==1:
                continue
            savepath=clustersavedir
            plotCTcomp(labels,ctlist,savepath,savenamecluster,addName)
            
def compMinibatchKmean(ctlist,nclusterL,n_pcs,addName=''):
    for ncluster in nclusterL:
        savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
        readpath=os.path.join(clustersavedir,savenamecluster)
        if not os.path.exists(readpath):
            print('DNE: '+readpath)
            continue
        with open(readpath, 'rb') as input:
            labels = pickle.load(input)
        labels=np.array(labels)
        if np.unique(labels).shape[0]==1:
            continue
            
        savepath=clustersavedir
        plotCTcomp(labels,ctlist,savepath,savenamecluster,addName)

In [9]:
# plot by cluster
# combine all exprs to one plot 
np.random.seed(seed)
for xcorr in plot_sample_X:
    exprs=None
    exprs_raw=None
    celltype_broad=None
    celltype_sub=None
    region=None
    samplenameList=None
    sobj_coord_np=None
    
    for s in plot_samples.keys():
        sampleidx=plot_samples[s]        
        samplename=s+'X_'+xcorr
        muplot=np.mean(featureslist[samplename][:,commstarmap],axis=1)
        muplot_raw=np.mean(featureslist[s+'X_'+'raw'][:,commstarmap],axis=1)
            
        if celltype_broad is None:
            exprs=muplot
            exprs_raw=muplot_raw
            celltype_broad=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']
            celltype_sub=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'cell_type_label']
            region=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']
            samplenameList=np.repeat(s,np.sum(scaleddata.obs['sample']==sampleidx))
            sobj_coord_np=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,['x','y']].to_numpy()
        else:
            exprs=np.concatenate((exprs,muplot))
            exprs_raw=np.concatenate((exprs_raw,muplot_raw))
            celltype_broad=np.concatenate((celltype_broad,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']),axis=None)
            celltype_sub=np.concatenate((celltype_sub,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'cell_type_label']),axis=None)
            region=np.concatenate((region,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']),axis=None)
            samplenameList=np.concatenate((samplenameList,np.repeat(s,np.sum(scaleddata.obs['sample']==sampleidx))),axis=None)
            sobj_coord_np=np.concatenate((sobj_coord_np,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,['x','y']].to_numpy()),axis=0)
       
    origCT=np.unique(celltype_broad)
    celltypeplot=np.concatenate((origCT,list(combineCelltype.keys())),axis=None)
    sampledir=os.path.join(plotsavepathModel,'combined'+xcorrModel)
#     if inverseAct:
    sampledir+='_beforeAct'
    clustersavedir=os.path.join(sampledir,'cluster')

    if 'leiden' in clustermethod:
        compLeidenPathway(celltype_broad,samplenameList,exprs,exprs_raw,n_neighbors,n_pcs,min_dist,resolution)
#     if 'dbscan' in clustermethod:
#         compDBscanPathway(celltype_broad,samplenameList,exprs,epslist,min_sampleslist,n_pcs)
#     if 'agglomerative' in clustermethod:
#         compAggPathway(celltype_broad,samplenameList,exprs,nclusterlist,aggMetric,n_pcs)
#     if 'kmeanbatch' in clustermethod:
#         compMinibatchKmeanPathway(celltype_broad,samplenameList,exprs,nclusterlist,n_pcs)
   

In [12]:
np.sum(muplot-muplot_raw)

-1311.0448